# Binary Classification: Multivariate Logistic Regression with Statsmodels

Predict whether a bank client is likely to default on their loan or not.

Note that interest rate indicates the 3-month interest rate between banks and duration indicates the time since the last contact was made with a given consumer. The previous variable shows whether the last marketing campaign was successful with this customer. The march and may are Boolean variables that account for when the call was made to the specific customer and credit shows if the customer has enough credit to avoid defaulting.

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Load data

In [2]:
# load bank csv data and store in df
url = "https://raw.githubusercontent.com/lucaskienast/Classification-Models/main/1)%20Binary%20Classification/Bank_data.csv"
df = pd.read_csv(url)
df = df.drop(["Unnamed: 0"], axis=1)
df.head()

,interest_rate,credit,march,may,previous,duration,y
0,1.334,0.0,1.0,0.0,0.0,117.0,no
1,0.767,0.0,0.0,2.0,1.0,274.0,yes
2,4.858,0.0,1.0,0.0,0.0,167.0,no
3,4.120,0.0,0.0,0.0,0.0,686.0,yes
4,4.856,0.0,1.0,0.0,0.0,157.0,no


## Map target: yes/no -> 1/0

In [3]:
# change yes/no in column y to 1/0
df["y"] = np.where(df["y"]=="yes", 1, 0)
df.head()

,interest_rate,credit,march,may,previous,duration,y
0,1.334,0.0,1.0,0.0,0.0,117.0,0
1,0.767,0.0,0.0,2.0,1.0,274.0,1
2,4.858,0.0,1.0,0.0,0.0,167.0,0
3,4.120,0.0,0.0,0.0,0.0,686.0,1
4,4.856,0.0,1.0,0.0,0.0,157.0,0


## Explore data

In [4]:
# show summary descriptive statistics
df.describe(include="all")

,interest_rate,credit,march,may,previous,duration,y
count,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000
mean,2.835776,0.034749,0.266409,0.388031,0.127413,382.177606,0.500000
std,1.876903,0.183321,0.442508,0.814527,0.333758,344.295990,0.500483
min,0.635000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000
25%,1.042750,0.000000,0.000000,0.000000,0.000000,155.000000,0.000000
50%,1.466000,0.000000,0.000000,0.000000,0.000000,266.500000,0.500000
75%,4.956500,0.000000,1.000000,0.000000,0.000000,482.750000,1.000000
max,4.970000,1.000000,1.000000,5.000000,1.000000,2653.000000,1.000000


## Declare features and targets

In [5]:
# create feature (X) and target (y) variables
y = df['y']
X1 = df[['interest_rate','credit','march','previous','duration']]

## Train Multivariate Logistic Regression Model

In [6]:
# build model
X = sm.add_constant(X1) # y-intercept
log_reg = sm.Logit(y, X)
log_results = log_reg.fit()

Optimization terminated successfully.
         Current function value: 0.336664
         Iterations 7


In [7]:
# show summary statistics
print(log_results.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  518
Model:                          Logit   Df Residuals:                      512
Method:                           MLE   Df Model:                            5
Date:                Fri, 27 Aug 2021   Pseudo R-squ.:                  0.5143
Time:                        19:14:51   Log-Likelihood:                -174.39
converged:                       True   LL-Null:                       -359.05
Covariance Type:            nonrobust   LLR p-value:                 1.211e-77
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.0211      0.311     -0.068      0.946      -0.631       0.589
interest_rate    -0.8001      0.089     -8.943      0.000      -0.975      -0.625
credit            2.3585      1.088     

Interpretation:

- given the all same circumstances (feature values) except the credit score (which ranges b/w 0 and 1 here), a client that has a credit score of 1 is 2.36 times as likely to pay back their loan as one with a credit score of 0.
- vice versa for the other features

## Model Accuracy

In [8]:
# show confusion matrix
log_results.pred_table()

array([[218.,  41.],
       [ 30., 229.]])

In [9]:
# confusion matrix formatted as df
cm_df = pd.DataFrame(log_results.pred_table())
cm_df.columns = ["Predicted 0", "Predicted 1"]
cm_df = cm_df.rename(index={0:"Actual 0", 1:"Actual 1"})
cm_df

,Predicted 0,Predicted 1
Actual 0,218.0,41.0
Actual 1,30.0,229.0


- For 218 observations, the model predicted 0 when the true value was 0.
- For 30 observations, the model predicted 0 when the true value was 1.
- vice versa for when the model predicted 1

In [10]:
# calculate actual accuracy
cm = np.array(cm_df)
accuracy_train = (cm[0,0] + cm[1,1])/cm.sum()
accuracy_train

0.862934362934363

The model shows 86% accuracy on the training data.

## Test Model

In [11]:
# load test bank csv data and store in df
url = "https://raw.githubusercontent.com/lucaskienast/Classification-Models/main/1)%20Binary%20Classification/Bank_data_testing.csv"
df_test = pd.read_csv(url)
df_test = df_test.drop(["Unnamed: 0"], axis=1)
df_test["y"] = np.where(df_test["y"]=="yes", 1, 0)
df_test.head()

,interest_rate,credit,march,may,previous,duration,y
0,1.313,0.0,1.0,0.0,0.0,487.0,0
1,4.961,0.0,0.0,0.0,0.0,132.0,0
2,4.856,0.0,1.0,0.0,0.0,92.0,0
3,4.120,0.0,0.0,0.0,0.0,1468.0,1
4,4.963,0.0,0.0,0.0,0.0,36.0,0


In [12]:
# declare features and targets of test dataset
y_test = df_test['y']
X1_test = df_test[['interest_rate','credit','march','previous','duration']]
X_test = sm.add_constant(X1_test) # y-intercept

In [13]:
def confusion_matrix(data,actual_values,model):
        # Confusion matrix 
        
        # Parameters
        # ----------
        # data: data frame or array
            # data is a data frame formatted in the same way as your input data (without the actual values)
            # e.g. const, var1, var2, etc. Order is very important!
        # actual_values: data frame or array
            # These are the actual values from the test_data
            # In the case of a logistic regression, it should be a single column with 0s and 1s
            
        # model: a LogitResults object
            # this is the variable where you have the fitted model 
            # e.g. results_log in this course
        # ----------
        
        #Predict the values using the Logit model
        pred_values = model.predict(data)
        # Specify the bins 
        bins=np.array([0,0.5,1])
        # Create a histogram, where if values are between 0 and 0.5 tell will be considered 0
        # if they are between 0.5 and 1, they will be considered 1
        cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
        # Calculate the accuracy
        accuracy = (cm[0,0]+cm[1,1])/cm.sum()
        # Return the confusion matrix and 
        return cm, accuracy

In [14]:
confusion_matrix(X_test, y_test, log_results)

(array([[93., 18.],
        [13., 98.]]), 0.8603603603603603)

The model performs slightly worse on the test data, which is expected.